<a href="https://colab.research.google.com/github/zdwhite/Thinkful-Unit-2/blob/master/Challenge_Feedback_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re as re

In [0]:
# Grab and process the raw data.
data_path = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/yelp_labelled.txt"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['message', 'sentiment']

In [0]:
sms_raw['message']=sms_raw['message'].str.lower()

In [0]:
#before we tackle the NB model Let's get a sense of some key words

#Split
good_sentiment=sms_raw[sms_raw['sentiment']==1]['message'].apply(lambda x:x.split())

#Looking through our bag of words, there are lots of punctuations and possibly special characters let's get rid of those
good_sentiment = pd.DataFrame(good_sentiment).apply(lambda x: pd.Series(x['message']),axis=1).stack().reset_index(level=1, drop=True).reset_index()

good_sentiment.columns=['Index','Words']

good_sentiment['Words']=good_sentiment['Words'].apply(lambda x: re.sub('[^A-Za-z0-9]+','',x))

GSWC = pd.DataFrame(good_sentiment['Words'].value_counts().reset_index())
GSWC['sentiment']='Good'

#Split
bad_sentiment=sms_raw[sms_raw['sentiment']==0]['message'].apply(lambda x:x.split())

bad_sentiment = pd.DataFrame(bad_sentiment).apply(lambda x: pd.Series(x['message']),axis=1).stack().reset_index(level=1, drop=True).reset_index()

bad_sentiment.columns=['Index','Words']

#Looking through our bag of words, there are lots of punctuations and possibly special characters let's get rid of those
bad_sentiment['Words']=bad_sentiment['Words'].apply(lambda x: re.sub('[^A-Za-z0-9]+','',x))

bad_sentiment['Words'].value_counts()

BSWC = pd.DataFrame(bad_sentiment['Words'].value_counts().reset_index())
BSWC['sentiment']='Bad'

Bag_of_words=GSWC.append(BSWC)
Bag_of_words.columns=['Word','Count','Sentiment']

In [0]:
f, ax = plt.subplots(figsize=(150, 30))
ax.set_xticklabels(labels = Bag_of_words['Word'],rotation=90)
ax = sns.barplot(x='Word',y='Count',hue='Sentiment',data=Bag_of_words[Bag_of_words['Count']>1])

In [0]:
#for a word to be a keyword of bad sentiment let's assume that we choose all words that
#appear greater in frequency as bad sentiment by at least 1 STD from the mean frequency of all words.
#also we should pluck words that only appear in bad sentiment phrases

#Summary Statistics
Bag_of_words.describe()
#std = Bag_of_words.describe()['Count'][2]




In [0]:
#Lets see if we need to normalize these two groups
#It appears that they are very similar in mean and standard deviation so no we shouldn't worry about normalization


print(Bag_of_words[Bag_of_words['Sentiment']=='Bad'].describe())

print(Bag_of_words[Bag_of_words['Sentiment']=='Good'].describe())

In [0]:

#pull list of keywords from Bag_of_Words

  
test = Bag_of_words[Bag_of_words["Sentiment"] == "Good"].join(Bag_of_words[Bag_of_words["Sentiment"] == "Bad"].set_index("Word"), on = "Word", lsuffix="_Good", rsuffix="_Bad")

test['Sentiment_Good'], test['Sentiment_Bad'] = 'Good', 'Bad'

test = test.set_index('Word')



In [0]:

test.fillna(0,inplace=True)
test

#Bag_of_words.describe()
std = Bag_of_words.describe()['Count'][2]

keyword = []
for word in test.index.values:
  #print(test[test['Word']==word])
  if test.loc[word][0] < test.loc[word][2]+std*2:
    keyword.append(word)
  if (test.loc[word][0]) == 0 & (test.loc[word][2] > 2):
    if word not in keyword:
      keyword.append(word)

In [0]:

keyword


In [0]:
sms_raw

In [0]:
# look for keywords with in the message
for key in keyword:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )
    

#Currently this throws an error because one of our keys contains a ")", this is obviously annoying    

In [0]:
#Run this once it will return True if bad sentiment and False if good, 
sms_raw['sentiment'] = (sms_raw['sentiment'] == 0)

sms_raw

In [0]:
f, ax = plt.subplots(figsize=(80, 80))
ax.set_xticklabels(labels = test.index.values,rotation=90)
ax = sns.heatmap(sms_raw.corr())

In [0]:
#eliminating keys if we wanted  to filter the correlation matrix by keywords that correlate highly with one another

cor_mat= pd.DataFrame(sms_raw.corr())
thresh = .75

cor_keywords = pd.DataFrame([],columns=['keyword','value','pair'])
for key1 in cor_mat.index.values:
  for key2 in cor_mat.index.values:
    if key1 != key2:
      if key1 not in cor_keywords['keyword']:
        if cor_mat.loc[key1][key2]> thresh:
          cor_keywords.loc[-1]= [key1,cor_mat.loc[key1][key2],key2]
          cor_keywords.index = cor_keywords.index+1
          
  
cor_keywords=cor_keywords.sort_index()
#cor_mat.loc['sentiment']['sentiment']
cor_keywords

After cleaning up a lot of our data with regular expression and case sensitivity we start to see some of the words that correlate fairly highly to one another.

In [0]:
sms_raw

data = sms_raw[keyword]
target = sms_raw['sentiment']

In [187]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

Number of mislabeled points out of a total 1000 points : 180


First pass shows us mislabeling  180 out of 1000, not too shabby.

Lets try and detect good sentiment instead and see if there is a difference

In [0]:
data_path = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/yelp_labelled.txt"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['message', 'sentiment']

In [0]:
sms_raw['message']=sms_raw['message'].str.lower()

In [0]:
test.fillna(0,inplace=True)
test

#Bag_of_words.describe()
std = Bag_of_words.describe()['Count'][2]

keyword = []
for word in test.index.values:
  #print(test[test['Word']==word])
  if test.loc[word][0]+std > test.loc[word][2]:
    keyword.append(word)
  if (test.loc[word][0]) >= 1 & (test.loc[word][2] == 0):
    if word not in keyword:
      keyword.append(word)

In [0]:
for key in keyword:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )

In [0]:
#Run this once it will return True if bad sentiment and False if good, 


sms_raw

In [0]:
sms_raw

data = sms_raw[keyword]
target = sms_raw['sentiment']

In [193]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

Number of mislabeled points out of a total 1000 points : 163


We were able to squeeze just a little more accuracy out of our model using good sentiment detection versus.

Let's see how it works with a new data set.

In [0]:

data_path = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/imdb_labelled.txt"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['message', 'sentiment']

sms_raw['message']=sms_raw['message'].str.lower()





In [0]:
for key in keyword:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )
    
sms_raw

In [0]:
sms_raw['sentiment'] = (sms_raw['sentiment'] != 0)

In [0]:
#keep the same good sentiment keywords as befor

data = sms_raw[keyword]
target = sms_raw['sentiment']

In [198]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}, {}% inaccuracy rate".format(
    data.shape[0],
    (target != y_pred).sum(),
    round(((target != y_pred).sum()/data.shape[0])*100,2)
))

Number of mislabeled points out of a total 748 points : 157, 20.99% inaccuracy rate


Our keywords from the yelp data set was able to correctly identify 79% of good sentitment IMDB ratings.

In [0]:
#let's check that last dataset and see how it performs 

#if i wasn't lazy i'd just make a function out of all of this and pass the dataset through for the results


data_path = ("https://raw.githubusercontent.com/zdwhite/Thinkful-Unit-2/master/Sentiment%20Data%20Sets/amazon_cells_labelled.txt"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['message', 'sentiment']

sms_raw['message']=sms_raw['message'].str.lower()





In [0]:
for key in keyword:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )
    
sms_raw

In [0]:
sms_raw['sentiment'] = (sms_raw['sentiment'] != 0)

In [0]:
#keep the same good sentiment keywords as befor

data = sms_raw[keyword]
target = sms_raw['sentiment']

In [204]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}, {}% inaccuracy rate".format(
    data.shape[0],
    (target != y_pred).sum(),
    round(((target != y_pred).sum()/data.shape[0])*100,2)
))

Number of mislabeled points out of a total 1000 points : 225, 22.5% inaccuracy rate


In [0]:
#What are the messages that are miss_labled and is there anything about them that is special.
miss_labeled = np.where(y_pred !=  target)[0]

miss_labeled

sms_raw.iloc[miss_labeled,:]